In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Load the dataset
df = pd.read_csv('Uncleaned_Ds_jobs.csv')

In [ ]:
# Initial dataset overview
print("Initial Data Overview:")
df.head()
df.info()
print(f"Missing Values:\n{df.isnull().sum()}")
print(f"Duplicated Rows: {df.duplicated().sum()}")




# Data Cleaning and Preparation




In [ ]:

# Clean Salary Estimate and create salary columns
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'[^\d\-]', '', regex=True)
df['Average Salary'] = df['Salary Estimate'].apply(
    lambda x: (int(x.split('-')[0]) + int(x.split('-')[1])) // 2 if '-' in x else int(x)
)
df[['Min Salary', 'Max Salary']] = df['Salary Estimate'].str.split('-', expand=True)
df['Min Salary'] = pd.to_numeric(df['Min Salary'], errors='coerce')
df['Max Salary'] = pd.to_numeric(df['Max Salary'], errors='coerce')

#Clean Salary Range column
df['Salary range'].isnull().sum()
df = df.dropna(subset=['Salary range'])
df['Salary range'].isnull().sum()

#Convert Salary range column into object
df['Salary range'] = df['Salary range'].astype(object)

#Clean state_full column
df['state_full'].isnull().sum()
df = df.dropna(subset=['state_full'])
df['state_full'].isnull().sum()

In [ ]:

# Extract state information from Location column
df['state'] = df['Location'].str.split(", ").str[-1].str.strip().str.title()


In [ ]:

# Map state abbreviations to full names
state_abbr_to_full = {
    'Ny': 'New York', 'Va': 'Virginia', 'Ma': 'Massachusetts', 'Ca': 'California', 
    'Il': 'Illinois', 'Mo': 'Missouri', 'Wa': 'Washington', 'Dc': 'District of Columbia', 
    'Tn': 'Tennessee', 'Tx': 'Texas', 'Pa': 'Pennsylvania', 'Az': 'Arizona', 
    'Wi': 'Wisconsin', 'Ga': 'Georgia', 'Fl': 'Florida', 'Ne': 'Nebraska', 
    'Ks': 'Kansas', 'Nh': 'New Hampshire', 'Nj': 'New Jersey', 'La': 'Louisiana', 
    'Oh': 'Ohio', 'In': 'Indiana', 'Md': 'Maryland', 'Co': 'Colorado', 
    'Ut': 'Utah', 'Or': 'Oregon', 'Mi': 'Michigan', 'Sc': 'South Carolina', 
    'Ms': 'Mississippi', 'Al': 'Alabama', 'Ri': 'Rhode Island', 'Ia': 'Iowa', 
    'Mn': 'Minnesota', 'Ok': 'Oklahoma', 'Ct': 'Connecticut', 'Nc': 'North Carolina', 
    'De': 'Delaware', 'Wv': 'West Virginia'
}
df['state_full'] = df['state'].map(state_abbr_to_full)


In [ ]:

# Standardize text columns
df['Job Title'] = df['Job Title'].str.strip().str.title()
df['Company Name'] = df['Company Name'].str.strip().str.title()


In [ ]:

# Remove irrelevant rows
df = df[~df['state'].isin(['Remote', 'United States'])]



# Feature Engineering


In [ ]:



# Extract key skills from job descriptions
def extract_keywords(description):
    keywords = ['Python', 'SQL', 'Machine Learning', 'Data Analysis', 'Tableau', 'Excel', 'Power BI']
    found = [kw for kw in keywords if isinstance(description, str) and kw in description]
    return ", ".join(found)

df['key_skills'] = df['Job Description'].apply(extract_keywords)


In [ ]:

# Create a salary range column
df['Salary range'] = pd.cut(
    df['Average Salary'], 
    bins=[50, 100, 150, 200, 271], 
    labels=["Low", "Mid", "High", "Very High"]
)



# Data Visualization



In [ ]:

# Salary distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['Average Salary'], kde=True, color="skyblue", bins=20)
plt.title('Salary Distribution')
plt.xlabel('Average Salary')
plt.ylabel('Frequency')
plt.show()


In [ ]:

# Top job titles
top_titles = df['Job Title'].value_counts().head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_titles.index, y=top_titles.values, palette="Blues_d")
plt.title('Top 10 Most Common Job Titles')
plt.xlabel('Job Title')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


In [ ]:

# Top states by job count
top_states = df['state_full'].value_counts().head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_states.index, y=top_states.values, palette="viridis")
plt.title('Top 10 States by Job Count')
plt.xlabel('State')
plt.ylabel('Job Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:

# Average salary by state
state_salary = df.groupby('state')['Average Salary'].mean().sort_values(ascending=False).head(10)
print("Top 10 States by Average Salary:")
print(state_salary)




# Reorder Columns


In [ ]:

column_order = [
    'Job Title', 'Job Description', 'key_skills',       
    'Salary Estimate', 'Average Salary', 'Salary range', 'Min Salary', 'Max Salary', 
    'Company Name', 'Location', 'state', 'state_full',  
    'Headquarters', 'Size', 'Founded', 'Sector', 'Revenue', 'Competitors',  
    'Rating'
]

df = df[column_order]


In [ ]:

# Final dataset overview
print("Final Data Overview:")
df.head()


In [ ]:
df.info()
